## Install Libraries

In [ ]:
!pip install torch transformers accelerate bitsandbytes -U
!pip install sentencepiece
!pip install gradio

## Import libraries

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import re
from google.colab import userdata
import gradio as gr
HUGGING_FACE_TOKEN = userdata.get("HUGGING_FACE_TOKEN")

In [ ]:
device = "cuda"

## Load Model

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", token=HUGGING_FACE_TOKEN,)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", quantization_config=bnb_config, token=HUGGING_FACE_TOKEN,)

## Setup function whill keep track of chat template and inversion

In [ ]:
messages = []

In [ ]:
def send_promptTo_MistralAI(question):
  global messages
  messages.append({'role': 'user', 'content': question})
  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)
  model.to(device)

  generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  response = decoded[0]
  matches = list(re.finditer(r'\[/INST\]', response))

  if matches:
      last_match = matches[-1]
      start = last_match.end()
      response = response[start:].strip()
  response = response.replace('</s>', '')
  messages.append({'role': 'assistant', 'content': response})
  return response


In [ ]:
def slow_echo(message, history):
    response = call_predict_api(message)
    for i in range(len(response)):
        time.sleep(0.05)
        yield f"{response[:i + 1]}"

def call_predict_api(message):
    try:
        response = send_promptTo_MistralAI(message)
        return response
    except requests.exceptions.RequestException as e:
        return f"Error getting reply: {e}"

demo = gr.ChatInterface(slow_echo)

if __name__ == "__main__":
    demo.launch()